In [1]:
#quiz 1
import torch
import torch.nn as nn
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

training_epochs = 15
batch_size = 100
root = './data'
mnist_train = dset.MNIST(root=root, train=True, transform=transforms.ToTensor(), download=True)
mnist_test = dset.MNIST(root=root, train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True,drop_last=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=True,drop_last=True)

In [7]:
#quiz 2
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
linear = torch.nn.Linear(784, 10, bias = True).to(device) #softmax
torch.nn.init.normal_(linear.weight) #weight linit

Parameter containing:
tensor([[ 0.2731,  0.3375,  0.1582,  ...,  0.7269,  0.8668,  0.1255],
        [ 0.0744,  0.6831,  0.3122,  ...,  0.7408, -0.1900,  1.6641],
        [ 1.0756,  1.5867, -0.0803,  ..., -1.3937,  0.2254,  0.8176],
        ...,
        [ 0.7233,  0.0213,  0.4726,  ..., -0.1498,  1.6609,  0.8554],
        [ 1.3822, -1.0026,  0.5326,  ..., -0.7408,  0.1800, -0.9022],
        [ 0.6672,  1.3645, -1.1123,  ..., -1.8661,  0.9046,  0.1023]],
       requires_grad=True)

In [8]:
#quiz 3
#Loss fn - cross Entropy Loss
criterion = torch.nn.CrossEntropyLoss().to(device)

#optimizer - SGD
optimizer = torch.optim.SGD(linear.parameters(), lr=0.1)

In [10]:
#quiz 4
for epoch in range(training_epochs):
    for i,(imgs,labels) in enumerate(train_loader): # X=imgs, Y=labels
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1,28*28).to(device)
        
        outputs = linear(imgs)
        loss = criterion(outputs,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        _,argmax = torch.max(outputs, 1)
        accuracy = (labels == argmax).float().mean()
        
        if (i+1)%100==0:
            print('Epoch[{}/{}], Step[{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1,training_epochs,i+1, len(train_loader), loss.item(), accuracy.item()*100))

Epoch[1/15], Step[100/600], Loss: 4.0898, Accuracy: 41.00%
Epoch[1/15], Step[200/600], Loss: 1.7256, Accuracy: 70.00%
Epoch[1/15], Step[300/600], Loss: 1.7750, Accuracy: 62.00%
Epoch[1/15], Step[400/600], Loss: 0.8882, Accuracy: 78.00%
Epoch[1/15], Step[500/600], Loss: 1.2919, Accuracy: 70.00%
Epoch[1/15], Step[600/600], Loss: 1.5846, Accuracy: 71.00%
Epoch[2/15], Step[100/600], Loss: 1.0821, Accuracy: 76.00%
Epoch[2/15], Step[200/600], Loss: 0.9890, Accuracy: 79.00%
Epoch[2/15], Step[300/600], Loss: 1.1451, Accuracy: 82.00%
Epoch[2/15], Step[400/600], Loss: 1.0064, Accuracy: 78.00%
Epoch[2/15], Step[500/600], Loss: 1.5315, Accuracy: 75.00%
Epoch[2/15], Step[600/600], Loss: 1.1116, Accuracy: 81.00%
Epoch[3/15], Step[100/600], Loss: 0.8846, Accuracy: 86.00%
Epoch[3/15], Step[200/600], Loss: 0.9137, Accuracy: 83.00%
Epoch[3/15], Step[300/600], Loss: 1.0880, Accuracy: 80.00%
Epoch[3/15], Step[400/600], Loss: 0.7421, Accuracy: 81.00%
Epoch[3/15], Step[500/600], Loss: 0.7415, Accuracy: 80.0

In [12]:
#quiz 5
#test
linear.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for i,(imgs,labels) in enumerate(test_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        imgs = imgs.view(-1,28*28)
        
        outputs = linear(imgs)
        
        _,argmax = torch.max(outputs, 1) #max()를 통해 최종 출력이 가장 높은 class 선택
        total += imgs.size(0)
        correct += (labels == argmax).sum().item()
    print('Test accuracy for {} images: {:.2f}%'.format(total, correct / total *100))

Test accuracy for 10000 images: 89.65%
